# 决策树

-  导入相关库

In [10]:
import numpy as np
import math
import operator
from graphviz import Digraph

- 建立一个简单的数据集

In [11]:
def ceratedata():
    dataset = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataset, labels

- 计算信息熵

当取自有限的样本时，熵的公式可以表示:
$$H(X)=-\sum_{i} P(x_{i})\log _{a}P(x_{i})$$


In [12]:
def calShannonEnt(dataset):
    numEnt = len(dataset)  # 获得list的长度 即实例总数   注（a）
    labelcounts = {}  # 创建一个字典，来存储数据集合中不同label的数量 如 dataset包含3 个‘no’  2个‘yes ’ （用键-值对来存储）
    for featVect in dataset:  # 对数据集合的每一个样本进行for遍历
        currentlabel = featVect[-1]  # 获得样本的标签
        if currentlabel not in labelcounts.keys():  # 如果当前标签在字典键值中不存在
            labelcounts[currentlabel] = 0  # 初值
        labelcounts[currentlabel] += 1  # 若已经存在该键所对应的值加1
    shannonEnt = 0.0
    for key in labelcounts:
        prob = float(labelcounts[key]) / numEnt
        shannonEnt -= prob * math.log(prob, 2)
    return shannonEnt

In [13]:
def splitdataset(dataset, axis, values):
    '''
    arguments:
        dataset:数据集
        axis:数据集的某一列
        values:分割数据的树节点
    return:
        retdataset:剩余数据集
        
    '''
    retdataset = []
    for featvect in dataset:
#         print('featvect:', featvect)
        if featvect[axis] == values:
            reducedfeatvec = featvect[:axis]
#             print('reducedfeatvec:', reducedfeatvec)
            reducedfeatvec.extend(featvect[axis+1:])
            retdataset.append(reducedfeatvec)
#             print('retdataset:', retdataset)
    print('retdataset:', retdataset)
    return retdataset

In [14]:
# 测试
dataset, lables = ceratedata()
splitdataset(dataset, 0, 0)
splitdataset(dataset, 0, 1)

retdataset: [[1, 'no'], [1, 'no']]
retdataset: [[1, 'yes'], [1, 'yes'], [0, 'no']]


[[1, 'yes'], [1, 'yes'], [0, 'no']]

- 选择最好的分割点

特征选择在于选择对数据有分类能力的特征，这样可以提高决策树的分类能力。通常特征选择的准则时信息增益,在选择特征时
，我们对所有特征都进行尝试，在分割完成后，计算所有情况下的信息增益，选取那个信息增益最大的所对应的特征作为树的分割节点。


就像我们上面的测试代码所显示的那样，在特征1下我们已经完成了数据分割，计算剩下数据集的信息熵，用它减去未分割前数据集的信息熵，这就是特征1的信息增益。接下来，同样的方式计算特征2的信息熵，两者进行对比即可。就可以得到最好的分割点了。

- 信息增益

特征集A对训练数据集D的信息增益为$g(D,A)$,定义为集合D的信息熵$H(D)$与特征A在给定条件D的经验条件熵$H(D|A)$之差，即
$$g(D,A)=H(D|A)-H(D)$$

In [15]:
def chooseBestFeatureToSplit(dataset):
    '''
    arguments:
        dataset:数据集
    return：
        bestfeature:返回要作为树节点的特征的列索引
    '''
    numfeatures = len(dataset[0])-1
    basentropy = calShannonEnt(dataset)
    bestimgain = 0.0
    bestfeature = -1
    for i in range(numfeatures):
        featlist = [example[i] for example in dataset]  # 用于将第i列的元素提取出来
        uniquevalu = set(featlist)  # 利用集合的特性删除掉重复的元素，神来之笔。
        newentropy = 0.0
        for value in uniquevalu:
            subdataset = splitdataset(dataset, i, value)
            prob = len(subdataset)/float(len(dataset))
            newentropy += prob*calShannonEnt(subdataset)
        imformationgain = basentropy-newentropy
        if(imformationgain>bestimgain):
            bestimgain = imformationgain
            bestfeature = i
    return bestfeature

- 投票机制

如果已经处理完了所有属性，但是类标签依然不是唯一的，此时需要我们决定如何处理该叶子节点，在这种情况下我们一般会采取投票机制。

  在若干次迭代后，假设数据集只剩下[[1, 'no'], [1, 'yes'], [0, 'no'], [1, 'yes']]
    如果再调用一次splitdataset(dataset, 0, 1)，数据集就只剩下['yes',‘yes’'no'], 需要投票表决选择类标签,返回出现次数最多的那个
    (事实上程序采取的算法是存在问题的，要是两个标签出现的次数相同，就会出现误差，其实我们可以采取某种加权的方式来解决这个问题)

In [16]:
def majorcnt(classlist):
    classcount={}
    for vote in classlist:
        if vote not in classcount.keys():
            classcount[vote] = 0
        classcount[vote] += 1
    sortedclasscount = sorted(classcount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedclasscount[0][0]


- 建立决策树

In [17]:
def creatree(dataset, labels):
    
    classlist = [example[-1] for example in dataset]  # classlist收集的是数据的最后一列
     #  以下应该是树终止的条件
    #  在我们把特征1作为最优的分割点后，在特征1为no的这一边，数据集只剩下[[1, 'no'],[1,'n0']]
    #   容易看出，类标签只剩下一类，满足下面的条件。
    #  count() 方法用于统计字符串里某个字符出现的次数。
    
    if classlist.count(classlist[0]) == len(classlist):
        
        return classlist[0]
    #  若是类标签并非只剩下一类，这是就要采用投票机制
    if len(dataset[0]) == 1:
        return majorcnt(classlist)
    # 在dataset数据集反复的使用chooseBestFeatureToSplit()来选取最好的内部节点
    # 因为函数是递归使用的，所以每次进入此函数的dataset是不同的
    bestfeat = chooseBestFeatureToSplit(dataset)
    bestfeatlabel = labels[bestfeat]  # 读取当前最好分类样本特征的名字
    
    mytree = {bestfeatlabel: {}}  # python创建树节点的方式
    del(labels[bestfeat])  # 删掉已经用来分类点样本特征
    featvalues = [example[bestfeat] for example in dataset]
    uniquevalue = set(featvalues)
    print(dataset)
    for value in uniquevalue:
        sublabels = labels[:]
        mytree[bestfeatlabel][value] = creatree(splitdataset(dataset, bestfeat, value), sublabels)
    return mytree

In [18]:
data, item = ceratedata()
tree = creatree(data, item)
print('tree:',tree)


retdataset: [[1, 'no'], [1, 'no']]
retdataset: [[1, 'yes'], [1, 'yes'], [0, 'no']]
retdataset: [[1, 'no']]
retdataset: [[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]
[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
retdataset: [[1, 'no'], [1, 'no']]
retdataset: [[1, 'yes'], [1, 'yes'], [0, 'no']]
retdataset: [['no']]
retdataset: [['yes'], ['yes']]
[[1, 'yes'], [1, 'yes'], [0, 'no']]
retdataset: [['no']]
retdataset: [['yes'], ['yes']]
tree: {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


In [23]:
def traves_tree(tree):
    for key, value in tree.items():
        print(key,":",value)
        
    return key,value
    
# def graphviz_tree(tree,Xlables,Ylables,X):  
#     key, value = traves_tree(tree)
#     if key == Xlables[:]:
#         dot.node('A', key)
    
    
    
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-23-f60605e2bfdb>, line 10)